# Pricing of Asian Options

Whereas the payoff of European options solely depends on the price of the underlying stock at the time of maturity, the payoff of Asian options depends on the price of the underlying asset at a predetermined discrete set of times between options' inception and maturity. This fixed set of dates is called monitoring dates. We consider only two types of Asian options in this notebook: those whose payoff is determined by the geometric mean of the price of the underlying asset at monitoring dates and those by the arithmetic mean. The former has a closed form analytic solution, whereas the latter does not (we will have to rely on Monte Carlo estimation as a result).

We assume that the stock prices follow Geometric Brownian Motion, i.e. the price $S_t$ at time $t$ of a stock can be expressed as
$$
S_t = S_0 \exp \left\{ \left( r - \frac{1}{2} \sigma^2 \right) t + \sigma W_t \right\}
$$
where $S_0$ is the initial stock price (at time 0), $r$ is the risk-free interest rate, $\sigma$ is the volatility of the stock, and $W_t$ is the standard Brownian motion.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats.mstats import gmean

print('numpy version: ',np.__version__)
print('pandas version: ',pd.__version__)

numpy version:  1.16.5
pandas version:  0.25.1


### Geometric Asian Options

A geometric Asian call option with strike price $K$ is a path-dependent option whose payoff at maturity $T$ is $(\bar{S}_G - K)^+$, where $\bar{S}_G$ is the geometric mean of the stock prices for a given set of monitoring dates $0 \leq t_1 < t_2 < \cdots < t_m \leq T$. The geometric mean of stock prices denoted by $\bar{S}_G$ is defined by
$$
\bar{S}_G = \left( \prod_{i=1}^{m} S_{t_i} \right) ^ {1 /m}
$$
It is fairly straighforward to see that $\log (\bar{S}_G) \sim N(\bar{\mu}, \bar{\sigma}^2)$, where
$$
\bar{\mu} = \log S_0 + \left(r - \frac{1}{2} \sigma^2 \right) \bar{t}, \quad \text{where} \quad \bar{t} = \frac{1}{m} \sum_{i=1}^m t_i
$$
$$
\bar{\sigma}^2 = \frac{\sigma^2}{m^2} \sum_{i=1}^m (2m - 2i + 1) t_i
$$
Therefore, the price of this option has the closed form analytic solution:
$$
\mathbb{E}[(\bar{S}_G - K)^+] = e^{-rT} \left( e^{\bar{\mu} + \frac{1}{2} \bar{\sigma}^2} \Phi(\bar{\sigma} - \theta) - K \Phi(-\theta) \right) \quad
\text{with} \quad \theta = \frac{1}{\bar{\sigma}} \left(\log K - \bar{\mu} \right)
$$
and $\Phi$ is the $CDF$ of standard normal.

In [2]:
# For simplicity we assume that the monitoring dates are at the end of each month

def BS_asian_geometric_call(S_0, r, sigma, K, T):
    '''
    Black-Scholes price for a geometric Asian call option:
    S_0 - price of the underlying stock at time 0
    r - annual risk-free interest rate in decimal
    sigma - volatility of the underlying stock
    K - strike price
    T - time to maturity in years
    The result is rounded to 5 decimal places
    '''
    t = np.floor(12 * T)
    t_bar = (t + 1) / 24
    mu_bar = np.log(S_0) + (r - 0.5 * sigma ** 2) * t_bar
    sigma_bar = np.sqrt(((sigma ** 2) / (t ** 2)) * 1 / 72 * t * (t + 1) * (2 * t + 1))
    theta = (1 / sigma_bar) * (np.log(K) - mu_bar)
    bs_asian_geometric_call_price = np.exp(-r * T) * (np.exp(mu_bar + 0.5 * sigma_bar ** 2) * norm.cdf(sigma_bar - theta) - K * norm.cdf(-theta))
    return round(bs_asian_geometric_call_price, 5)

In [3]:
# For simplicity we assume that the monitoring dates are at the end of each month

def MC_asian_geometric_call(S_0, r, sigma, K, T, n):
    '''
    Price estimate for geometric Asian call option using Monte Carlo simulation:
    We will generate paths for the Geometric Brownian Motion
    S_0 - price of the underlying stock at time 0
    r - annual risk-free interest rate in decimal
    sigma - volatility of the underlying stock
    K - strike price
    T - time to maturity in years
    n - number of iterations
    Returns both the price and standard error
    The results are  rounded to 5 decimal places
    '''
    t = np.floor(12 * T)
    Z = (r - 0.5 * sigma **2) * (1/12) + sigma * (1 / np.sqrt(12)) * np.random.standard_normal(size=(n, int(t)))
    W = np.cumsum(Z, axis=1) 
    Y = S_0 * np.exp(W)
    X1 = gmean(Y, axis=1)
    X = np.exp(-r * T) * np.maximum(X1 - K, np.zeros(n))   
    asian_geometric_call_price = np.mean(X)
    asian_geometric_call_se = np.std(X) / np.sqrt(n)
    return round(asian_geometric_call_price, 5), round(asian_geometric_call_se, 5)

### Comparison of the Analytic Price and Monte Carlo Estimate

We will now compare the Analytic price of call options and their Monte Carlo estimates for various strike prices and various sample sizes. 

#### Geometric Asian Call Option with strike price $K=40$,  $S_0 = 50$, $r=0.05$, $\sigma=0.2$, and $T=1$.

In [4]:
np.random.seed(2019)
sample_size = [1e4, 1e5, 1e6, 1e7]
mc_estimates = []
mc_se = []
bs_price = np.repeat(BS_asian_geometric_call(50, 0.05, 0.2, 40, 1), 4)

for n in sample_size:
    mc_call = MC_asian_geometric_call(50, 0.05, 0.2, 40, 1, int(n))
    mc_estimates.append(mc_call[0])
    mc_se.append(mc_call[1])

df = pd.DataFrame({'Sample Size': sample_size, 'Black-Scholes Price': bs_price, 
                   'Monte Carlo Estimate': mc_estimates, 'Standard Error': mc_se})
df

,Sample Size,Black-Scholes Price,Monte Carlo Estimate,Standard Error
0,10000.0,10.69977,10.79074,0.05927
1,100000.0,10.69977,10.71293,0.01867
2,1000000.0,10.69977,10.70506,0.00592
3,10000000.0,10.69977,10.69618,0.00187


#### Geometric Asian Call Option with strike price $K=60$,  $S_0 = 50$, $r=0.05$, $\sigma=0.2$, and $T=1$.

In [5]:
np.random.seed(2019)
sample_size = [1e4, 1e5, 1e6, 1e7]
mc_estimates = []
mc_se = []
bs_price = np.repeat(BS_asian_geometric_call(50, 0.05, 0.2, 60, 1), 4)

for n in sample_size:
    mc_call = MC_asian_geometric_call(50, 0.05, 0.2, 60, 1, int(n))
    mc_estimates.append(mc_call[0])
    mc_se.append(mc_call[1])

df = pd.DataFrame({'Sample Size': sample_size, 'Black-Scholes Price': bs_price, 
                   'Monte Carlo Estimate': mc_estimates, 'Standard Error': mc_se})
df

,Sample Size,Black-Scholes Price,Monte Carlo Estimate,Standard Error
0,10000.0,0.29875,0.31994,0.01370
1,100000.0,0.29875,0.29660,0.00419
2,1000000.0,0.29875,0.30073,0.00134
3,10000000.0,0.29875,0.29843,0.00042


#### Observation:
If one increases the sample size in a Monte Carlo estimation by a factor of $n$, the standard error will be reduced by a factor of $\sqrt n$. One can easily observe this phenomennon by comparing the 1st and the 3rd rows (or 2nd and 4th for that matter) in each dataframe above by noting the 100-fold increase in sample size and 10-fold decrease in standard error.

### Arithmetic Asian Options

An arithmetic Asian call option with strike price $K$ is a path-dependent option whose payoff at maturity $T$ is $(\bar{S}_A - K)^+$, where $\bar{S}_A$ is the arithmetic mean of the stock prices for a given set of monitoring dates $0 \leq t_1 < t_2 < \cdots < t_m \leq T$. The arithmetic mean of stock prices denoted by $\bar{S}_A$ is defined by:
$$
\bar{S}_A = \frac{1}{m} \left( \sum_{i=1}^{m} S_{t_i} \right)
$$
Unlike geometric Asian options, arithmetic Asian options do not have an analytic formula for pricing / valuation; therefore, we will have to estimate them. Again, we will use Monte Carlo for that purpose.

In [6]:
# For simplicity we assume that the monitoring dates are at the end of each month

def MC_asian_arithmetic_call(S_0, r, sigma, K, T, n):
    '''
    Price estimate for arithmetic Asian call option using Monte Carlo simulation:
    We will generate paths for the Geometric Brownian Motion
    S_0 - price of the underlying stock at time 0
    r - annual risk-free interest rate in decimal
    sigma - volatility of the underlying stock
    K - strike price
    T - time to maturity in years
    n - number of iterations
    Returns both the price and standard error
    The results are  rounded to 5 decimal places
    '''
    t = np.floor(12 * T)
    Z = (r - 0.5 * sigma **2) * (1/12) + sigma * (1 / np.sqrt(12)) * np.random.standard_normal(size=(n, int(t)))
    W = np.cumsum(Z, axis=1) 
    Y = S_0 * np.exp(W)
    X1 = np.mean(Y, axis=1)
    X = np.exp(-r * T) * np.maximum(X1 - K, np.zeros(n))   
    asian_arithmetic_call_price = np.mean(X)
    asian_arithmetic_call_se = np.std(X) / np.sqrt(n)
    return round(asian_arithmetic_call_price, 5), round(asian_arithmetic_call_se, 5)

#### Arithmetic Asian Call Option with strike price $K=40$,  $S_0 = 50$, $r=0.05$, $\sigma=0.2$, and $T=1$.

In [7]:
np.random.seed(2019)
sample_size = [1e4, 1e5, 1e6, 1e7]
mc_estimates = []
mc_se = []
bs_price = np.repeat(np.nan, 4)

for n in sample_size:
    mc_call = MC_asian_arithmetic_call(50, 0.05, 0.2, 40, 1, int(n))
    mc_estimates.append(mc_call[0])
    mc_se.append(mc_call[1])

df = pd.DataFrame({'Sample Size': sample_size, 'Black-Scholes Price': bs_price, 
                   'Monte Carlo Estimate': mc_estimates, 'Standard Error': mc_se})
df

,Sample Size,Black-Scholes Price,Monte Carlo Estimate,Standard Error
0,10000.0,NaN,10.95334,0.06009
1,100000.0,NaN,10.87266,0.01891
2,1000000.0,NaN,10.86475,0.00599
3,10000000.0,NaN,10.85595,0.00189


#### Arithmetic Asian Call Option with strike price $K=60$,  $S_0 = 50$, $r=0.05$, $\sigma=0.2$, and $T=1$.

In [8]:
np.random.seed(2019)
sample_size = [1e4, 1e5, 1e6, 1e7]
mc_estimates = []
mc_se = []
bs_price = np.repeat(np.nan, 4)

for n in sample_size:
    mc_call = MC_asian_arithmetic_call(50, 0.05, 0.2, 60, 1, int(n))
    mc_estimates.append(mc_call[0])
    mc_se.append(mc_call[1])

df = pd.DataFrame({'Sample Size': sample_size, 'Black-Scholes Price': bs_price, 
                   'Monte Carlo Estimate': mc_estimates, 'Standard Error': mc_se})
df

,Sample Size,Black-Scholes Price,Monte Carlo Estimate,Standard Error
0,10000.0,NaN,0.36156,0.01499
1,100000.0,NaN,0.33461,0.00457
2,1000000.0,NaN,0.33939,0.00146
3,10000000.0,NaN,0.33706,0.00046


#### Observation:
If one increases the sample size in a Monte Carlo estimation by a factor of $n$, the standard error will be reduced by a factor of $\sqrt n$. One can easily observe this phenomennon by comparing the 1st and the 3rd rows (or 2nd and 4th for that matter) in each dataframe above by noting the 100-fold increase in sample size and 10-fold decrease in standard error.